In [1]:
import numpy as np
import matplotlib.pyplot as plt
import accelerator_calculations as ac 
from accelerator_calculations import pq

# scSPS Cavity Calculations

In [2]:
proton = ac.particle(1*pq.m_p, 1*pq.e, 1*pq.c)

scsps_beam = ac.beam(particle = proton,
                     total_energy = 1.3 *pq.TeV)

scsps = ac.accelerator(beam = scsps_beam,
                       circumference = 6.9 * pq.km,
                       harmonic_number = 1,
                       max_dipole_field = 6*pq.T)

## RF Frequency Decision

Revolution frequency and period is

In [3]:
print('{:.2f~}'.format(scsps.revolution_frequency().to(pq.kHz)))

43.45 kHz


In [4]:
print('{:.2f~}'.format((scsps.revolution_period()).to(pq.us)))

23.02 us


at least one gap is required for beam protection systems. These won't change much over the acceleration cycle, split the beam up into two bunch trains and two 3.5$\mu$s gaps. We need bunch spacings to be 25 ns so the harmonic number could be

In [5]:
bunch_spacing = 25*pq.ns
temp_h = np.ceil(((scsps.revolution_period())/(bunch_spacing)).to(pq.dimensionless))
print("Harmonic Number = {}".format(temp_h))
scsps.update_harmonic_number(temp_h)
print("RF Frequency = {:.2f~}".format(scsps.rf_frequency().to(pq.MHz)))

number_of_bunches = 20*pq.us / bunch_spacing
print(number_of_bunches.to(pq.dimensionless))

Harmonic Number = 921.0 dimensionless
RF Frequency = 40.02 MHz
800.0 dimensionless


In this case, were we to use a simple pillbox the required radius would be

In [6]:
cav = ac.cavity(scsps.rf_frequency())
print('radius = {:.2f~}'.format(cav.radius_for_tm101()))

radius = 2.87 m


This is too large, increase the harmonic number by a factor of 12

In [7]:
new_h = temp_h*12
scsps.update_harmonic_number(new_h)
print("Harmonic Number = {}".format(new_h))
print("RF Frequency = {:.2f~}".format(scsps.rf_frequency().to(pq.MHz)))

number_of_bunches = 16*pq.us / bunch_spacing
print("Number of Bunches = {:.0f~}".format(number_of_bunches.to(pq.dimensionless)))

Harmonic Number = 11052.0 dimensionless
RF Frequency = 480.19 MHz
Number of Bunches = 640


In [8]:
cav = ac.cavity(scsps.rf_frequency())
print('radius = {:.2f~}'.format(cav.radius_for_tm101()))

radius = 0.24 m


This is more reasonable and would allow smaller bunches for potential FCC upgrades.

## Cavity Bandwidth
As the particles are accelerated their revolution frequency, $f_{rev}$ will change,
$$
f_{rev} = \frac{C}{\beta c},
$$
where $C$ is the circumference of the accelerator. Since $\beta$ is expected to change, there will be a finite  bandwidth which is set by the injection and extraction energy, the particle type and the harmonic number.

In [9]:
print('bandwidth = {:.3f~}'.format(scsps.cavity_bandwidth(injection_energy = 26 *pq.GeV,
                                                          extraction_energy = 1.3*pq.TeV).to(pq.kHz)))

bandwidth = 312.652 kHz


Resonant superconducting cavities have quality factors on the order of $10^{10}$, the bandwidth of such cavities is

In [10]:
cav.bandwidth(1.5e10).to(pq.Hz)

<Quantity(0.03201261239451657, 'hertz')>

The superconducting cavity doesn't have the required bandwidth, which will cause a phase error. Assuming that the velocity stays constant over the phase error will be the fractional part of the actual revolution period multiplied by the angular RF frequency.

In [11]:
temp= ((scsps.revolution_period(total_energy=26*pq.GeV) * (cav.frequency())).to(pq.dimensionless))
print(temp)

11059.200663276253 dimensionless


In [12]:
num_periods = divmod(temp, 1)

if num_periods[1] > 0.5:
    phase_error = (num_periods[0]+1 - temp)*2*np.pi
else:
    phase_error = num_periods[1]*2*np.pi*pq.radian
    
print("Phase error = {:.2f~}".format(phase_error.to(pq.degree)))

Phase error = 72.24 deg


In [13]:
print("Number of turns for 90 degree shift = {:.0f~}".format(np.pi*pq.radian/4/phase_error))

Number of turns for 90 degree shift = 1


## Required Cavity Gap Voltage

The total acceleration cycle must be completed in around one minute, some additional time could be needed to turn down the current on the SC magnets, so assume $T_{\textrm{acc}} = 30 \,\textrm{s}$. In this time the particle energy will change from 26 GeV to 1.3 TeV, but the revolution period will change only slightly, thus the total number of turns can be estimated from
$$
N_{\textrm{turn}} = \frac{T_{acc}}{T_{rev}}.
$$
To get a "worst case scenario" we can use the revolution period at the injection energy, there will necessarily be fewer turns than this as beta will increase slightly.

In [14]:
T_acc = 30*pq.s  # acceleration time

In [15]:
num_turns = np.ceil((T_acc/scsps.revolution_period(total_energy=26*pq.GeV)).to(pq.dimensionless))
print("Num Turns = {:.0e~}".format(num_turns))

Num Turns = 1e+06


In [16]:
energy_swing = 1.3*pq.TeV - 26*pq.GeV
energy_per_turn = (energy_swing/num_turns).to(pq.MeV)

print("Energy Gain per Turn = {:.2f~}".format(energy_per_turn))

Energy Gain per Turn = 0.98 MeV


In [17]:
(energy_per_turn/(1*pq.e)).to(pq.MV)

<Quantity(0.9780461909494939, 'megavolt')>

If the cavity length is approx 30 cm this gives a gap field of

In [19]:
(energy_per_turn/(1*pq.e)/(30*pq.cm)).to(pq.MV/pq.m)

<Quantity(3.260153969831646, 'megavolt / meter')>